In [2]:
# Imports & settings
import pandas as pd

In [3]:
# Load the business.csv file as a DataFrame called businesses
businesses = pd.read_csv("datasets/businesses.csv")
# print(businesses.head())

# Sort businesses from oldest businesses to youngest
sorted_businesses = businesses.sort_values(by="year_founded")

# Display the first few lines of sorted_businesses
# sorted_businesses.head()
sorted_businesses[:10]

,business,year_founded,category_code,country_code
64,Kongō Gumi,578,CAT6,JPN
94,St. Peter Stifts Kulinarium,803,CAT4,AUT
107,Staffelter Hof Winery,862,CAT9,DEU
106,Monnaie de Paris,864,CAT12,FRA
103,The Royal Mint,886,CAT12,GBR
111,Sean's Bar,900,CAT4,IRL
112,Marinelli Bell Foundry,1040,CAT12,ITA
96,Affligem Brewery,1074,CAT9,BEL
102,Munke Mølle,1135,CAT12,DNK
58,Ma Yu Ching's Bucket Chicken House,1153,CAT4,CHN


## 1. The oldest businesses in North America
<p>We see that Kongō Gumi is the world's oldest continuously operating business, beating out the second oldest business by well over 100 years! It's a little hard to read the country codes, though. Wouldn't it be nice if we had a list of country names to go along with the country codes?</p>


In [4]:
# Load countries.csv to a DataFrame
countries = pd.read_csv("datasets/countries.csv")
display(countries.head())

# Merge sorted_businesses with countries
businesses_countries = sorted_businesses.merge(countries, on="country_code")

# Filter businesses_countries to include countries in North America only
north_america = businesses_countries[
    businesses_countries["continent"] == "North America"
]
north_america.head()

,country_code,country,continent
0,AFG,Afghanistan,Asia
1,AGO,Angola,Africa
2,ALB,Albania,Europe
3,AND,Andorra,Europe
4,ARE,United Arab Emirates,Asia


,business,year_founded,category_code,country_code,country,continent
22,La Casa de Moneda de México,1534,CAT12,MEX,Mexico,North America
28,Shirley Plantation,1638,CAT1,USA,United States,North America
33,Hudson's Bay Company,1670,CAT17,CAN,Canada,North America
35,Mount Gay Rum,1703,CAT9,BRB,Barbados,North America
40,Rose Hall,1770,CAT19,JAM,Jamaica,North America


## 2. The oldest business on each continent
<p>Now we can see that the oldest company in North America is La Casa de Moneda de México, founded in 1534. Why stop there, though, when we could easily find out the oldest business on every continent? </p>

In [5]:
# Create continent, which lists only the continent and oldest year_founded
continent = pd.DataFrame(
    businesses_countries.groupby("continent")["year_founded"].min(),
    columns=["year_founded"],
)
display(continent)

# Merge continent with businesses_countries
merged_continent = continent.merge(businesses_countries, on="year_founded")
display(merged_continent)

# Subset continent so that only the four columns of interest are included
subset_merged_continent = merged_continent[
    ["continent", "country", "business", "year_founded"]
]
display(subset_merged_continent)

,year_founded
continent,
Africa,1772
Asia,578
Europe,803
North America,1534
Oceania,1809
South America,1565


,year_founded,business,category_code,country_code,country,continent
0,1772,Mauritius Post,CAT16,MUS,Mauritius,Africa
1,578,Kongō Gumi,CAT6,JPN,Japan,Asia
2,803,St. Peter Stifts Kulinarium,CAT4,AUT,Austria,Europe
3,1534,La Casa de Moneda de México,CAT12,MEX,Mexico,North America
4,1809,Australia Post,CAT16,AUS,Australia,Oceania
5,1565,Casa Nacional de Moneda,CAT3,PER,Peru,South America


,continent,country,business,year_founded
0,Africa,Mauritius,Mauritius Post,1772
1,Asia,Japan,Kongō Gumi,578
2,Europe,Austria,St. Peter Stifts Kulinarium,803
3,North America,Mexico,La Casa de Moneda de México,1534
4,Oceania,Australia,Australia Post,1809
5,South America,Peru,Casa Nacional de Moneda,1565


## 3. Unknown oldest businesses
<p>BusinessFinancing.co.uk wasn't able to determine the oldest business for some countries, and those countries are simply left off of <code>businesses.csv</code> and, by extension, <code>businesses</code>. However, the <code>countries</code> that we created <em>does</em> include all countries in the world, regardless of whether the oldest business is known. </p>
<p>We can compare the two datasets in one DataFrame to find out which countries don't have a known oldest business! </p>

In [14]:
# Create a merged dataFrame, all_countries 
all_countries = pd.merge(businesses, countries, how="outer")
# print(all_countries.head())
# print all the rows that contain null values
# print(all_countries[all_countries.isnull().any(axis=1)])

# Filter to include only countries without oldest businesses
missing_countries = all_countries[all_countries["year_founded"].isnull()]
# print(missing_countries)

# Create a series of the country names with missing oldest business data
missing_countries_series = missing_countries["country"]


# Display the series
pd.DataFrame(missing_countries_series)

,country
163,Angola
164,Antigua and Barbuda
165,Bahamas
166,Dominican Republic
167,Ecuador
168,Fiji
169,"Micronesia, Federated States of"
170,Ghana
171,Gambia
172,Grenada


## 4. Adding new oldest business data
<p>It looks like we've got some holes in our dataset! Fortunately, we've taken it upon ourselves to improve upon BusinessFinancing.co.uk's work and find oldest businesses in a few of the missing countries. We've stored the newfound oldest businesses in <code>new_businesses</code>

In [7]:
# Import new_businesses.csv
new_businesses = pd.read_csv("datasets/new_businesses.csv")

# Add the data in new_businesses to the existing businesses
# print(businesses)
# print(new_businesses)
all_businesses = pd.concat([new_businesses, businesses])
# all_businesses = businesses.append(new_businesses) # other way to combine both data frames
display(all_businesses)

# Merge and filter to find countries with missing business data
new_all_countries = all_businesses.merge(countries, how="outer")
new_missing_countries = new_all_countries[new_all_countries["business"].isnull()]
# print(new_missing_countries)

# Group by continent and create a "count_missing" column
# count_missing = pd.DataFrame(new_missing_countries.groupby('continent')['country'].count())   # other way to perform aggregate
count_missing = new_missing_countries.groupby("continent").agg({"country": "count"})
count_missing.columns = ["count_missing"]
# count_missing=count_missing.rename('count_missing')      #other way to rename the column
count_missing

,business,year_founded,category_code,country_code
0,Fiji Times,1869,CAT13,FJI
1,J. Armando Bermúdez & Co.,1852,CAT9,DOM
0,Hamoud Boualem,1878,CAT11,DZA
1,Communauté Électrique du Bénin,1968,CAT10,BEN
2,Botswana Meat Commission,1965,CAT1,BWA
...,...,...,...,...
158,Cafe Brasilero,1877,CAT4,URY
159,Hacienda Chuao,1660,CAT11,VEN
160,Australia Post,1809,CAT16,AUS
161,Bank of New Zealand,1861,CAT3,NZL


,count_missing
continent,
Africa,3
Asia,7
Europe,2
North America,5
Oceania,10
South America,3


## 5. The oldest industries
<p>Let's look at the most popular categories.</p>





In [8]:
# Import categories.csv and merge to businesses
categories = pd.read_csv("datasets/categories.csv")
businesses_categories = businesses.merge(categories, on="category_code")
# print(businesses_categories.head())


# Create a DataFrame which lists the number of oldest businesses in each category
count_business_cats = businesses_categories.groupby("category").agg(
    {"business": "count"}
)
# print(count_business_cats)

# Rename column and display the first five rows of the DataFrame
count_business_cats.columns = ["count"]
display(count_business_cats.head())

,count
category,
Agriculture,6
Aviation & Transport,19
Banking & Finance,37
"Cafés, Restaurants & Bars",6
Conglomerate,3


## 6. Restaurant representation
<p>No matter how we measure it, looks like Banking and Finance is an excellent industry to be in if longevity is our goal! Let's zoom in on another industry: cafés, restaurants, and bars. Which restaurants in our dataset have been around since before the year 1800?</p>

In [9]:
# Filter using .query() for CAT4 businesses founded before 1800; sort results
old_restaurants = businesses_categories.query(
    'year_founded < 1800 and category_code == "CAT4"'
)
# old_restaurants = businesses_categories[(businesses_categories['year_founded'] <1800) & (businesses_categories['category_code'] == 'CAT4')] # Other way to find same result
# print(old_restaurants)

# Sort the DataFrame
old_restaurants = old_restaurants.sort_values(by="year_founded")
old_restaurants

,business,year_founded,category_code,country_code,category
142,St. Peter Stifts Kulinarium,803,CAT4,AUT,"Cafés, Restaurants & Bars"
143,Sean's Bar,900,CAT4,IRL,"Cafés, Restaurants & Bars"
139,Ma Yu Ching's Bucket Chicken House,1153,CAT4,CHN,"Cafés, Restaurants & Bars"


## 7. Categories and continents
<p>St. Peter Stifts Kulinarium is old enough that the restaurant is believed to have served Mozart - and it would have been over 900 years old even when he was a patron! Let's finish by looking at the oldest business in each category of commerce for each continent. </p>

In [10]:
# Merge all businesses, countries, and categories together
businesses_categories_countries = businesses.merge(
    categories, on="category_code"
).merge(countries, on="country_code")
# print(businesses_categories_countries)

# Sort businesses_categories_countries from oldest to most recent
businesses_categories_countries = businesses_categories_countries.sort_values(
    by="year_founded", ascending=True
)
# print(businesses_categories_countries)

# Create the oldest by continent and category DataFrame
oldest_by_continent_category = businesses_categories_countries.groupby(
    ["continent", "category"]
).agg({"year_founded": "min"})
oldest_by_continent_category

year_founded
continent     category                                       
Africa        Agriculture                                1947
              Aviation & Transport                       1854
              Banking & Finance                          1892
              Distillers, Vintners, & Breweries          1933
              Energy                                     1968
              Food & Beverages                           1878
              Manufacturing & Production                 1820
              Media                                      1943
              Mining                                     1962
              Postal Service                             1772
Asia          Agriculture                                1930
              Aviation & Transport                       1858
              Banking & Finance                          1830
              Cafés, Restaurants & Bars                  1153
              Conglomerate                               1841
              Construction                                578
              Defense                                    1808
              Distillers, Vintners, & Breweries          1853
              Energy                                     1928
              Food & Beverages                           1820
              Manufacturing & Production                 1736
              Media                                      1931
              Mining                                     1913
              Postal Service                             1800
              Retail                                     1883
              Telecommunications                         1885
              Tourism & Hotels                           1584
Europe        Agriculture                                1218
              Banking & Finance                          1606
              Cafés, Restaurants & Bars                   803
              Consumer Goods                             1649
              Defense                                    1878
              Distillers, Vintners, & Breweries           862
              Manufacturing & Production                  864
              Media                                      1999
              Medical                                    1422
              Mining                                     1248
              Postal Service                             1520
              Telecommunications                         1912
              Tourism & Hotels                           1230
North America Agriculture                                1638
              Aviation & Transport                       1870
              Banking & Finance                          1891
              Distillers, Vintners, & Breweries          1703
              Food & Beverages                           1920
              Manufacturing & Production                 1534
              Media                                      1909
              Retail                                     1670
              Tourism & Hotels                           1770
Oceania       Banking & Finance                          1861
              Postal Service                             1809
South America Banking & Finance                          1565
              Cafés, Restaurants & Bars                  1877
              Defense                                    1811
              Food & Beverages                           1660
              Manufacturing & Production                 1621